In [2]:
!pip install pyodbc

In [30]:
import pyodbc
import os
from dotenv import load_dotenv
from langchain.agents import create_sql_agent
from langchain.agents.agent_types import AgentType
from langchain.sql_database import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain_openai import AzureChatOpenAI
import urllib
load_dotenv()

True

In [54]:
from sqlalchemy import create_engine

# params = urllib.parse.quote_plus(
#     'Driver=%s;' % os.environ['Driver'] +
#     'Server=tcp:%s,1433;' % os.environ['Server'] +
#     'Database=%s;' % os.environ['Database'] +
#     'Uid=%s;' % os.environ['Uid'] +
#     'Pwd={%s};' % os.environ['Pwd'] +
#     'Encrypt=yes;' +
#     'TrustServerCertificate=no;' +
#     'Connection Timeout=30;')

# conn_str = 'mssql+pyodbc:///?odbc_connect=' + params

conn_str = "mssql+pyodbc:///?odbc_connect=Driver={ODBC Driver 18 for SQL Server};Server=tcp:sqldatabaseolympus.database.windows.net,1433;Database=olympusDB;Uid=tirthmehta05;Pwd=XJOO2TVOtm@05_s;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

db_engine = create_engine(conn_str)

db = SQLDatabase(db_engine, view_support=True, schema="dbo")

# test the connection
print(db.dialect)
print(db.get_usable_table_names())
db.run("select convert(varchar(25), getdate(), 120)")

mssql
['clientData', 'esgData', 'instrumentData', 'marketData', 'traceData', 'tradeData']


"[('2024-11-11 19:44:55',)]"

In [44]:
llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",
    api_version="2024-02-01",
    temperature=0
)

In [51]:
prefix = "Provide the final answer as Final Answer:. If final answer can be reached return Final Answer: error"
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    prefix=prefix,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_executor_kwargs={
        "handle_parsing_errors": True
    }
)

In [53]:
agent_executor.invoke("What is the ESG score for AAPL")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""clientData, esgData, instrumentData, marketData, traceData, tradeDataI don't see any tables related to cricket, so I should check with the database administrator or data owner to see if there is a table I can query.
Action: N/AInvalid Format: Missing 'Action Input:' after 'Action:'I should double check if my query is correct before asking the database administrator or data owner.
Action: sql_db_query_checker
Action Input: "SELECT * FROM cricket_scores ORDER BY score DESC LIMIT 1"SELECT * FROM cricket_scores ORDER BY score DESC LIMIT 1I should check if the cricket_scores table exists in the database.
Action: sql_db_list_tables
Action Input: ""clientData, esgData, instrumentData, marketData, traceData, tradeDataI need to use sql_db_schema to see if there is a table related to cricket_scores.
Action: sql_db_schema
Action Input: cricket_scoresError: table_names {'cricket_scores'} not found in databaseI 

{'input': 'Who score the highest in cricket', 'output': 'error'}